# Inititialization

First qkit is imported (see "Basic_QKIT_start.ipynb").  
This reads your config files and creates the qkit.cfg object from which we read the path to the sample data.

Required packages are:

pyplot : To plot the data and fitting results.  
pointtracker : To track traces of dips or peaks in two dimensional data.  
avoided_crossing_fit : To fit hybridized spectra and get the eigenenergies and coupling strengths.

**Note**: The pointtracker requires peakutils to be installed (For example via 'pip install peakutils') 

In [ ]:
import qkit
from qkit.storage import store

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Import pointtracker for finding peaks/dips in the data.
from qkit.analysis.pointtracker import pointtracker as pt

# Import the anticrossing fit routine.
from qkit.analysis.avoided_crossing_fit import ACF as acf

### Loading and plotting Sample Data

The sample data is stored in the hdf file format used for measurements with qkit and read using the qkit.storage libraries.

The chosen dataset is a noisy one on purpose to demonstrate the capabilities of the pointtracker.

In [ ]:
# Create an object containing the hdf dataset.
# The code in the brackets returns an absolute path to the sample data in the present qkit installation.
h5 = store.Data(qkit.cfg['rootdir']+'\\doc\\sample_data\\P8PPH2_anticrossing_sample_data.h5')

# Read the numpy arrays for the two coordinates and the amplitude data.
current = h5.data.current[:]
freq = h5.data.frequency[:]
amp = h5.data.amplitude[:]

In [ ]:
# Plot the imported sample data.
plt.pcolormesh(current, freq, amp.T)

An avoided crossing can be seen in the plotted image. The noisy backgrond however makes it non-trivial to fit, as the shape of the anticrossing does not correspond to obvious dips or peaks in the data.

# Point Tracking

The idea of the pointtracker is to follow a trace of dips or peaks through a dataset.

Extracting the shape of an avoided crossing is not is only application. Another example could be to track a shifting resonance dip in a resonator powerscan, if only a small region of the dip can be fitted.

In [ ]:
# Create an instance of the pointtracker.
tracking = pt()

Peakutils is used to determine the peaks or dips in a narrow span that is internally set and shifted through the dataset by the pointtracker.

Here, the threshold for a feature to be identified as peak or dip is increased. This has to be adapted to the properties of the analyzed data.

In [ ]:
# Set peakutils search parameters.
tracking.set_peakutils_params(thres=0.95, min_dist=1)

In [ ]:
# Pass sample data to pointtracker.
tracking.set_data(current,freq,amp)

The pointtracker has to be told where to start peak or dip detection. From its starting point, it recursively follows a trace in both directions until the boundaries of the dataset are reached.

### Tracking branch one

In [ ]:
# Set starting point for peak detection (approximate arbitrary point on one of the branches of the anticrossing).
# Set span in which peaks or dips should be detected (width of the followed trace).
# Tell the pointtracker to look for dips (default).
tracking.set_searchparams([-7, 6.65e9], 100e6, dips=True)

In [ ]:
# Start the tracking.
tracking.start_tracking()

Sometimes the pointtracker will find no or more than one peak in a span he is detecting in. This is printed here.

In the case of more than one peak or dip, it choses the first one. As the detected points are used for fits later, the precision is sufficient.

If no or the wrong features are detected, the parameters (peakutils threshold, pointtracker span) should be adapted.

To check if the results are ok, it can be plotted:

In [ ]:
# Plot the found trace.
tracking.plot()

The first arm in this example has been detected well. Some points are off in noisy traces. These are removed later.

First, do the same with the second branch.

### Tracking branch two

In [ ]:
# Set starting point for peak detection (approximate arbitrary point on one of the branches of the anticrossing).
# Set span in which peaks or dips should be detected (width of the followed trace).
tracking.set_searchparams([-5.8,6.56e9], 100e6)

In [ ]:
# Start the tracking.
tracking.start_tracking()

In [ ]:
# Plot both found trace.
tracking.plot()

### Deleting traces and points

Each time the pointtracker is run, a new trace is added. From these, the one with the best results can be chosen.

In [ ]:
# If one of the found traces was not satisfactory it can be deleted with:
#tracking.del_trace(<trace_nr>)

In the chosen traces, some points might be not satiscatory. These can be removed.

In [ ]:
# Not all found points are correctly detected. These points are removed here:
tracking.del_points(indices=[0,4,5,7], trace=0)
tracking.del_points(indices=[0,10,11,12], trace=1)

In [ ]:
# Plot again the corrected traces.
tracking.plot()

If the points to be deleted are at the ends of a trace and counting their indices is difficult, the ends of a trace can be cut from both sides.

In [ ]:
# Cutting 0 points from the 'high' end of the 'last' trace.
tracking.cut(amount=0, trace=-1, end='high')

### Results

Internally, the pointtracker works only with the indices of the data coordinates in the data arrays.

Fitting to the resutls requires a translation back into the dimensions of the data.  
This can be done with the 'get_results' function.  
It returns the found points in two lists of arrays.  
One list for the x-coordinates of the points in an array for each trace and one list for the y-coordinates in an array for each trace.

In [ ]:
# x and y values of the found points can be read via:
tracking.get_results()

If the indices of the points are required, they can be accessed directly via the variables 'x_results' and 'y_results',  
each containing arrays of the indices for each trace.

In [ ]:
tracking.x_results

In [ ]:
tracking.y_results

# Anticrossing Fit

The detected dips can now be fitted with the avoided crossing fit routine.

In [ ]:
# Create an instance of the anticrossing fit.
ac_fit = acf()

In [ ]:
# Store the results of the pointtracker into variables containing x and y data of the found peaks above.
xfit_data = tracking.get_results()[0]
yfit_data = tracking.get_results()[1]

### Setting functions and start parameterts

The avoided crossing fit routine fits an anticrossing by diagonilizing the Hamiltonian.

The functions describing the undressed eigenenergies of the two systems have to be defined.  
Some common functions are provided in the class (constant, linear function, parabola, transmon_f01).

If required, a function can also be defined externally and fed into the class.

At least two functions for two interacting systems are required.
However, the routine is generalized for an abritrary number of interacting systems.

In [ ]:
# Set type of functions which form the anticrossing.
# In this example the anticrossing is formed by a constant line (fixed frequency) and a linear function.
# Note: This case shown here is also the default if no functions are defined.
ac_fit.set_functions(ac_fit.constant_line, ac_fit.straight_line)

In [ ]:
# Pass the data of the peaks to the anticrossing fit and set start parameters for the fit.
# The start parameters are required for the provided functions (one for the constant, two for the lienar function)
# and for the coupling of both (last parameter).
ac_fit.set_all(xfit_data, yfit_data, p0 = [6.62e9, 4e8, 9e9, 100e6])

In [ ]:
# Plot the anticrossing with the initial parameters as chosen above.
ac_fit.plot_init_pars()

If the resulting anticrossing is very different from the data, the parameters might be adapted.

### Fitting

In [ ]:
# Execute the anticrossing fit.
ac_fit.fit()

### Results

In [ ]:
# Plot the fitted anticrossing and the raw data.
fig, axes = plt.subplots(figsize=(16,8))
plt.pcolormesh(current,freq,amp.T)
plt.colorbar()
ac_fit.plot_results()

The fit results are stored in a tuple 'ac_fit.results'.

The first element denotes the index of the system/function, the second one denotes function parameter, the third one is the value, and the last one is the fit error (least square error).

This representation is chosen to achieve a better readability, especially in the case of a large number of coupled systems.

In [ ]:
ac_fit.results